In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc=SparkContext()

In [3]:
import json

In [4]:
rdd=sc.textFile("/home/deepika/Downloads/netflix_titles.json")

In [5]:
from pyspark.storagelevel import StorageLevel
rdd.persist(StorageLevel.MEMORY_ONLY)

/home/deepika/Downloads/netflix_titles.json MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
data= rdd.map(lambda x: json.loads(x))

In [7]:
data.take(1)

[{'show_id': 's1',
  'type': 'TV Show',
  'title': '3%',
  'director': '',
  'cast': 'João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi',
  'country': 'Brazil',
  'date_added': 'August 14, 2020',
  'release_year': '2020',
  'rating': 'TV-MA',
  'duration': '4 Seasons',
  'listed_in': 'International TV Shows, TV Dramas, TV Sci-Fi & Fantasy',
  'description': 'In a future where the elite inhabit an island paradise far from the crowded slums, you get one chance to join the 3% saved from squalor.'}]


# how many titles actor/actress appeared in?

In [8]:
titles= data.flatMap(lambda x: (x['cast'].split(",")))

In [9]:
gap=titles.filter(lambda x: x !='')

In [10]:
gap.collect()

['João Miguel',
 ' Bianca Comparato',
 ' Michel Gomes',
 ' Rodolfo Valente',
 ' Vaneza Oliveira',
 ' Rafael Lozano',
 ' Viviane Porto',
 ' Mel Fronckowiak',
 ' Sergio Mamberti',
 ' Zezé Motta',
 ' Celso Frateschi',
 'Demián Bichir',
 ' Héctor Bonilla',
 ' Oscar Serrano',
 ' Azalia Ortiz',
 ' Octavio Michel',
 ' Carmen Beato',
 'Tedd Chan',
 ' Stella Chung',
 ' Henley Hii',
 ' Lawrence Koh',
 ' Tommy Kuan',
 ' Josh Lai',
 ' Mark Lee',
 ' Susan Leong',
 ' Benjamin Lim',
 'Elijah Wood',
 ' John C. Reilly',
 ' Jennifer Connelly',
 ' Christopher Plummer',
 ' Crispin Glover',
 ' Martin Landau',
 ' Fred Tatasciore',
 ' Alan Oppenheimer',
 ' Tom Kane',
 'Jim Sturgess',
 ' Kevin Spacey',
 ' Kate Bosworth',
 ' Aaron Yoo',
 ' Liza Lapira',
 ' Jacob Pitts',
 ' Laurence Fishburne',
 ' Jack McGee',
 ' Josh Gad',
 ' Sam Golzari',
 ' Helen Carey',
 ' Jack Gilpin',
 'Erdal Beşikçioğlu',
 ' Yasemin Allen',
 ' Melis Birkan',
 ' Saygın Soysal',
 ' Berkan Şal',
 ' Metin Belgin',
 ' Ayça Eren',
 ' Selin Ulu

In [11]:
result= gap.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [12]:
result.collect()

[(' Vaneza Oliveira', 1),
 (' Viviane Porto', 1),
 (' Mel Fronckowiak', 1),
 (' Celso Frateschi', 1),
 (' Héctor Bonilla', 1),
 (' Octavio Michel', 1),
 (' Carmen Beato', 3),
 ('Tedd Chan', 1),
 (' Henley Hii', 5),
 (' Lawrence Koh', 1),
 (' Tommy Kuan', 1),
 (' Susan Leong', 2),
 (' Christopher Plummer', 8),
 (' Martin Landau', 4),
 (' Alan Oppenheimer', 1),
 (' Tom Kane', 3),
 ('Jim Sturgess', 2),
 (' Kate Bosworth', 4),
 (' Helen Carey', 2),
 (' Jack Gilpin', 1),
 (' Melis Birkan', 3),
 (' Berkan Şal', 2),
 (' Metin Belgin', 1),
 (' Selin Uludoğan', 1),
 ('Amina Khalil', 1),
 (' Ahmed Dawood', 5),
 (' Mahmoud Hijazi', 1),
 (' Tara Emad', 3),
 (' John Heard', 4),
 (' Clifton Collins Jr.', 5),
 (' Raayo S. Bakhirta', 2),
 (' Greeva Kansara', 1),
 (' Adah Sharma', 3),
 (' Indraneil Sengupta', 6),
 (' Anjori Alagh', 1),
 (' Vipin Sharma', 13),
 (' Shri Vallabh Vyas', 2),
 (' Molly Parker', 5),
 (' Dylan Schmid', 1),
 (" Brian d'Arcy James", 4),
 (' Maciej Musiał', 1),
 (' Michalina Olsz

In [13]:
import os
os.environ["PYTHONHASHSEED"]=str(232)

In [14]:
result.lookup(' Héctor Bonilla')

[]

# How many movies were released in 2008?

In [15]:
movies=data.filter(lambda x:x['release_year']=='2008')

In [16]:
movies.count()

125

In [17]:
movies.collect()

[{'show_id': 's5',
  'type': 'Movie',
  'title': '21',
  'director': 'Robert Luketic',
  'cast': 'Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, Jacob Pitts, Laurence Fishburne, Jack McGee, Josh Gad, Sam Golzari, Helen Carey, Jack Gilpin',
  'country': 'United States',
  'date_added': 'January 1, 2020',
  'release_year': '2008',
  'rating': 'PG-13',
  'duration': '123 min',
  'listed_in': 'Dramas',
  'description': 'A brilliant group of students become card-counting experts with the intent of swindling millions out of Las Vegas casinos by playing blackjack.'},
 {'show_id': 's10',
  'type': 'Movie',
  'title': '1920',
  'director': 'Vikram Bhatt',
  'cast': 'Rajneesh Duggal, Adah Sharma, Indraneil Sengupta, Anjori Alagh, Rajendranath Zutshi, Vipin Sharma, Amin Hajee, Shri Vallabh Vyas',
  'country': 'India',
  'date_added': 'December 15, 2017',
  'release_year': '2008',
  'rating': 'TV-MA',
  'duration': '143 min',
  'listed_in': 'Horror Movies, International Movies,

# List all the movies whose duration is greater than 100 mins ?

In [18]:
pairs = data.map(lambda x: (int(x['duration'].split(" ")[0]), x['title']))

In [19]:
pairs.collect()

[(4, '3%'),
 (93, '7:19'),
 (78, '23:59'),
 (80, '9'),
 (123, '21'),
 (1, '46'),
 (95, '122'),
 (119, '187'),
 (118, '706'),
 (143, '1920'),
 (103, '1922'),
 (1, '1983'),
 (1, '1994'),
 (89, '2,215'),
 (91, '3022'),
 (149, 'Oct-01'),
 (1, 'Feb-09'),
 (144, '22-Jul'),
 (124, '15-Aug'),
 (87, "'89"),
 (110, '\u200b\u200bKuch Bheege Alfaaz'),
 (128, '\u200bGoli Soda 2'),
 (117, '\u200bMaj Rati \u200b\u200bKeteki'),
 (100, '\u200bMayurakshi'),
 (2, '\u200bSAINT SEIYA: Knights of the Zodiac'),
 (84, '(T)ERROR'),
 (1, '(Un)Well'),
 (99, '#Alive'),
 (95, '#AnneFrank - Parallel Stories'),
 (1, '#blackAF'),
 (90, '#cats_the_mewvie'),
 (102, '#FriendButMarried'),
 (104, '#FriendButMarried 2'),
 (99, '#realityhigh'),
 (105, '#Roxy'),
 (56, '#Rucker50'),
 (125, '#Selfie'),
 (119, '#Selfie 69'),
 (1, 'แผนร้ายนายเจ้าเล่ห์'),
 (81, '¡Ay, mi madre!'),
 (97, 'Çarsi Pazar'),
 (106, 'Ég man þig'),
 (99, 'Çok Filim Hareketler Bunlar'),
 (107, 'Òlòtūré'),
 (93, 'Æon Flux'),
 (1, 'Şubat'),
 (89, '1 Chance 2

In [20]:
duration_above_hundred= pairs.filter(lambda x: x[0] >100)

In [21]:
duration_above_hundred.collect()

[(123, '21'),
 (119, '187'),
 (118, '706'),
 (143, '1920'),
 (103, '1922'),
 (149, 'Oct-01'),
 (144, '22-Jul'),
 (124, '15-Aug'),
 (110, '\u200b\u200bKuch Bheege Alfaaz'),
 (128, '\u200bGoli Soda 2'),
 (117, '\u200bMaj Rati \u200b\u200bKeteki'),
 (102, '#FriendButMarried'),
 (104, '#FriendButMarried 2'),
 (105, '#Roxy'),
 (125, '#Selfie'),
 (119, '#Selfie 69'),
 (106, 'Ég man þig'),
 (107, 'Òlòtūré'),
 (105, '1 Mile to You'),
 (109, '10,000 B.C.'),
 (109, '100 Meters'),
 (101, '100% Halal'),
 (101, '13TH'),
 (113, '14 Blades'),
 (102, '16 Blocks'),
 (102, '17 Again'),
 (114, '18 Presents'),
 (130, '1898: Our Last Men in the Philippines'),
 (143, '2 States'),
 (107, '2015 Dream Concert'),
 (119, '20th Century Women'),
 (101, "2307: Winter's Dream"),
 (140, '25 Kille'),
 (135, '26 Years'),
 (104, '28 Days'),
 (117, '3 Days to Kill'),
 (121, '3 Deewarein'),
 (124, '3 Heroines'),
 (164, '3 Idiots'),
 (114, '365 Days'),
 (116, '37 Seconds'),
 (105, '4L'),
 (120, '4th Republic'),
 (106, '5 C

In [22]:
duration_above_hundred.count()

2430

In [23]:
duration_above_hundred=duration_above_hundred.map(lambda x: x[1])

In [24]:
duration_above_hundred.collect()

['21',
 '187',
 '706',
 '1920',
 '1922',
 'Oct-01',
 '22-Jul',
 '15-Aug',
 '\u200b\u200bKuch Bheege Alfaaz',
 '\u200bGoli Soda 2',
 '\u200bMaj Rati \u200b\u200bKeteki',
 '#FriendButMarried',
 '#FriendButMarried 2',
 '#Roxy',
 '#Selfie',
 '#Selfie 69',
 'Ég man þig',
 'Òlòtūré',
 '1 Mile to You',
 '10,000 B.C.',
 '100 Meters',
 '100% Halal',
 '13TH',
 '14 Blades',
 '16 Blocks',
 '17 Again',
 '18 Presents',
 '1898: Our Last Men in the Philippines',
 '2 States',
 '2015 Dream Concert',
 '20th Century Women',
 "2307: Winter's Dream",
 '25 Kille',
 '26 Years',
 '28 Days',
 '3 Days to Kill',
 '3 Deewarein',
 '3 Heroines',
 '3 Idiots',
 '365 Days',
 '37 Seconds',
 '4L',
 '4th Republic',
 '5 Cowok Jagoan',
 '5CM',
 '6 Underground',
 '6-5=2',
 '7 Din Mohabbat In',
 '7 Khoon Maaf',
 '90 ML',
 '93 Days',
 'A Bad Moms Christmas',
 'A Beautiful Life',
 'A Billion Colour Story',
 'A Bridge Too Far',
 'A California Christmas',
 'A Choo',
 'A Clockwork Orange',
 'A Dangerous Woman',
 'A Fall from Grace

# List movies played by “Kareena Kapoor” ?

In [25]:
kareena = data.map(lambda x: (x['cast'].split(",")[0], x['title']))

In [26]:
kareena.collect()

[('João Miguel', '3%'),
 ('Demián Bichir', '7:19'),
 ('Tedd Chan', '23:59'),
 ('Elijah Wood', '9'),
 ('Jim Sturgess', '21'),
 ('Erdal Beşikçioğlu', '46'),
 ('Amina Khalil', '122'),
 ('Samuel L. Jackson', '187'),
 ('Divya Dutta', '706'),
 ('Rajneesh Duggal', '1920'),
 ('Thomas Jane', '1922'),
 ('Robert Więckiewicz', '1983'),
 ('', '1994'),
 ('Artiwara Kongmalai', '2,215'),
 ('Omar Epps', '3022'),
 ('Sadiq Daba', 'Oct-01'),
 ('Shahd El Yaseen', 'Feb-09'),
 ('Anders Danielsen Lie', '22-Jul'),
 ('Rahul Pethe', '15-Aug'),
 ('Lee Dixon', "'89"),
 ('Geetanjali Thapa', '\u200b\u200bKuch Bheege Alfaaz'),
 ('Samuthirakani', '\u200bGoli Soda 2'),
 ('Adil Hussain', '\u200bMaj Rati \u200b\u200bKeteki'),
 ('Soumitra Chatterjee', '\u200bMayurakshi'),
 ('Bryson Baugus', '\u200bSAINT SEIYA: Knights of the Zodiac'),
 ('', '(T)ERROR'),
 ('', '(Un)Well'),
 ('Yoo Ah-in', '#Alive'),
 ('Helen Mirren', '#AnneFrank - Parallel Stories'),
 ('Kenya Barris', '#blackAF'),
 ('', '#cats_the_mewvie'),
 ('Adipati Dolke

In [27]:
kareena1=[]
kareenaa= kareena.groupByKey()
for t in kareenaa.collect():
    kareena1.append((t[0],[i for i in t[1]]))

In [28]:
kareena1

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('',
  ['1994',
   '(T)ERROR',
   '(Un)Well',
   '#cats_the_mewvie',
   '#Rucker50',
   '100 Days Of Solitude',
   "100 Years: One Woman's Fight for Justice",
   '13TH',
   '14 Minutes from Earth',
   '3 Seconds Divorce',
   '60 Days In',
   '7 Days Out',
   '72 Cutest Animals',
   'A 3 Minute Hug',
   'A Gray State',
   'A Leaf of Faith',
   'A Life of Speed: The Juan Manuel Fangio Story',
   'A Lion in the House',
   'A Love Song for Latasha',
   'A Murder in the Park',
   'A new Capitalism',
   'A Perfect Crime',
   'A Secret Love',
   'A Tale of Two Kitchens',
   'A Week in Watts',
   'Abducted in Plain Sight',
   'Adhugo',
   'Afflicted',
   'After Maria',
   'After Porn Ends',
   'After Porn Ends 3',
   'After the Raid',
   'Age of Tanks',
   'Ajaibnya Cinta',
   'Alarmoty in the Land of Fire',
   'Alias JJ, la celebridad del mal',
   'Alien Worlds',
   'Alive and Kicking

In [29]:
movie_list_by_kareena_kapoor=sc.parallelize(kareena1)

In [30]:
movie_list_by_kareena_kapoor= movie_list_by_kareena_kapoor.filter(lambda x: x[0]!='')

In [31]:
movie_list_by_kareena_kapoor.collect()

[('Tedd Chan', ['23:59']),
 ('Jim Sturgess', ['21', 'Kidnapping Mr. Heineken']),
 ('Amina Khalil', ['122']),
 ('Omar Epps', ['3022']),
 ('Sadiq Daba', ['Oct-01']),
 ('Anders Danielsen Lie', ['22-Jul']),
 ('Rahul Pethe', ['15-Aug']),
 ('Soumitra Chatterjee', ['\u200bMayurakshi', 'Peace Haven']),
 ('Yoo Ah-in', ['#Alive', 'Burning', 'Chicago Typewriter']),
 ('Helen Mirren',
  ['#AnneFrank - Parallel Stories',
   "Legend of the Guardians: The Owls of Ga'Hoole",
   'The Hundred-Foot Journey',
   'Winchester']),
 ('Adipati Dolken',
  ['#FriendButMarried',
   '#FriendButMarried 2',
   'Hunter in the Blue Side of Manchester']),
 ('Flavia Hojda', ['#Selfie']),
 ('Maia Morgenstern', ['#Selfie 69']),
 ('Estefanía de los Santos', ['¡Ay, mi madre!']),
 ('Erdem Yener', ['Çarsi Pazar']),
 ('Charlize Theron',
  ['Æon Flux', 'Dark Places', 'The Old Guard', 'Young Adult']),
 ('Lexi Giovagnoli', ['1 Chance 2 Dance', "All Hallows' Eve"]),
 ('Steven Strait', ['10,000 B.C.']),
 ('Doh Kyung-soo', ['100 Days

In [32]:
movie_list_by_kareena_kapoor.lookup("Kareena Kapoor")

[['Chameli', 'Ek Main Aur Ekk Tu', 'Heroine', 'Kurbaan', 'Satyagraha']]